In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

<img src='2AFC_events.png' width=1200 height=1200/>

In [ ]:
behav_df = pd.DataFrame()
behav_df['Rewarded'] = [1, 1, 0]
behav_df['CorrectChoice'] = [1, 0, 0]
behav_df['CompletedTrial'] = [1, 1, 0]
behav_df["ChosenDirection"] = [1, 2, -1]
behav_df['TrialNumber'] = [0, 1, 2]
behav_df['WaitingTime'] = [3.1401, 1.0460, 2.2079]
behav_df.loc[:, 'TrialStartAligned'] = [110.187, 122.253, 135.293]  # seconds from beginning of recording (only really care about difference)

behav_df['DV'] = [-0.15556, 0.948718, -0.2]
behav_df['stim_dir'] = [1, 0, 1]

behav_df['CatchTrial'] = [False, False, False]

behav_df['PokeCenterStart'] = [6.4965, 4.5105, 8.4341]
behav_df['StimulusOnset'] = [6.7186, 4.9032, 8.6681]
behav_df['StimulusOffset'] = [7.0687, 5.2533, 9.0182]

behav_df['ResponseStart'] = [7.3495, 5.4722, 9.3196]
behav_df['ResponseEnd'] = [10.9949, 8.6123, 10.3656]

In [ ]:
behav_df.head()